# **Step 1: Install and Import Required Libraries**
In this section, we ensure that all required libraries are installed and imported.
- We use `scikit-fuzzy` for the fuzzy control system.
- We use `matplotlib` for graph visualization.
- We use `ipywidgets` for interactive controls.


In [ ]:
# Installing Required Libraries
!pip install -U scikit-fuzzy matplotlib ipywidgets --quiet

# Importing Libraries
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from skfuzzy import control as ctrl
import ipywidgets as widgets
from IPython.display import display, clear_output

print("✅ Project setup completed.")


#  Section 2: Defining Fuzzy Variables
We define the fuzzy variables (input and output) for the control system:
- Vibration (0 - 100 mm/s)
- Temperature (0 - 120 °C)
- Operation Time (0 - 500 hours)
- Risk Level (0 - 10)

In [ ]:
# Defining Fuzzy Variables
vibration = ctrl.Antecedent(np.arange(0, 101, 0.1), 'vibration')
temperature = ctrl.Antecedent(np.arange(0, 121, 1), 'temperature')
operation_time = ctrl.Antecedent(np.arange(0, 501, 1), 'operation_time')
risk_level = ctrl.Consequent(np.arange(0, 11, 0.1), 'risk_level')

# Membership functions for Vibration
vibration['low'] = fuzz.trimf(vibration.universe, [0, 0, 30])
vibration['medium'] = fuzz.trapmf(vibration.universe, [20, 40, 60, 80])
vibration['high'] = fuzz.trapmf(vibration.universe, [70, 90, 100, 100])

# Membership functions for Temperature
temperature['low'] = fuzz.trimf(temperature.universe, [0, 0, 40])
temperature['medium'] = fuzz.trapmf(temperature.universe, [30, 50, 70, 90])
temperature['high'] = fuzz.trapmf(temperature.universe, [80, 100, 120, 120])

# Membership functions for Operation Time
operation_time['short'] = fuzz.trapmf(operation_time.universe, [0, 0, 100, 150])
operation_time['moderate'] = fuzz.trapmf(operation_time.universe, [100, 200, 300, 400])
operation_time['long'] = fuzz.trapmf(operation_time.universe, [350, 400, 500, 500])

# Membership functions for Risk Level
risk_level['low'] = fuzz.trimf(risk_level.universe, [0, 0, 3])
risk_level['moderate'] = fuzz.trapmf(risk_level.universe, [2, 4, 6, 8])
risk_level['high'] = fuzz.trimf(risk_level.universe, [7, 10, 10])

print("✅ Fuzzy variables defined.")


# Section 3: Defining Fuzzy Rules and Controller
We define the rules that will determine the risk level based on the input values.



In [ ]:
# High Risk (Dominant)
rule1 = ctrl.Rule(vibration['high'] & temperature['high'] & operation_time['long'], risk_level['high'])
rule2 = ctrl.Rule(vibration['high'] & temperature['high'] & operation_time['moderate'], risk_level['high'])
rule3 = ctrl.Rule(vibration['high'] & temperature['medium'] & operation_time['long'], risk_level['high'])
rule4 = ctrl.Rule(vibration['medium'] & temperature['high'] & operation_time['long'], risk_level['high'])
rule5 = ctrl.Rule(vibration['high'] & temperature['medium'] & operation_time['moderate'], risk_level['high'])
rule6 = ctrl.Rule(vibration['medium'] & temperature['high'] & operation_time['moderate'], risk_level['high'])


# Moderate Risk (Balanced)
rule7 = ctrl.Rule(vibration['medium'] & temperature['medium'] & operation_time['moderate'], risk_level['moderate'])
rule8 = ctrl.Rule(vibration['medium'] & temperature['medium'] & operation_time['long'], risk_level['moderate'])
rule9 = ctrl.Rule(vibration['low'] & temperature['high'] & operation_time['long'], risk_level['moderate'])
rule10 = ctrl.Rule(vibration['high'] & temperature['low'] & operation_time['long'], risk_level['moderate'])
rule11 = ctrl.Rule(vibration['high'] & temperature['medium'] & operation_time['short'], risk_level['moderate'])
rule12 = ctrl.Rule(vibration['medium'] & temperature['low'] & operation_time['long'], risk_level['moderate'])
rule13 = ctrl.Rule(vibration['medium'] & temperature['high'] & operation_time['short'], risk_level['moderate'])
rule14 = ctrl.Rule(vibration['low'] & temperature['medium'] & operation_time['long'], risk_level['moderate'])

# Low Risk (Safe Conditions)
rule15 = ctrl.Rule(vibration['low'] & temperature['low'] & operation_time['short'], risk_level['low'])
rule16 = ctrl.Rule(vibration['low'] & temperature['low'] & operation_time['moderate'], risk_level['low'])
rule17 = ctrl.Rule(vibration['low'] & temperature['medium'] & operation_time['short'], risk_level['low'])
rule18 = ctrl.Rule(vibration['medium'] & temperature['low'] & operation_time['short'], risk_level['low'])
rule19 = ctrl.Rule(vibration['low'] & temperature['medium'] & operation_time['moderate'], risk_level['low'])
rule20 = ctrl.Rule(vibration['medium'] & temperature['low'] & operation_time['moderate'], risk_level['low'])

# Adjusting the Rules to Fix Test 5
'''
rule21 = ctrl.Rule(vibration['high'] & temperature['low'] & operation_time['moderate'], risk_level['moderate'])
rule22 = ctrl.Rule(vibration['high'] & temperature['low'] & operation_time['short'], risk_level['moderate'])

# Recreating the Control System
risk_control = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8,
    rule9, rule10, rule11, rule12, rule13, rule14, rule15,
    rule16, rule17, rule18, rule19, rule20,
    rule21, rule22
])
risk_simulation = ctrl.ControlSystemSimulation(risk_control)
'''

# Creating the Fuzzy Control System

risk_control = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6,
    rule7, rule8, rule9, rule10, rule11, rule12, rule13, rule14,
    rule15, rule16, rule17, rule18, rule19, rule20
])
risk_simulation = ctrl.ControlSystemSimulation(risk_control)

print("✅ Fuzzy control system created.")



# Section 4: Risk Calculation and Visualization
We define a clean function that calculates the risk level based on the inputs.

In [ ]:
def calculate_risk(vibration_value, temperature_value, time_value):
    print("\n🔧 Calculating Risk Level...")
    print(f"📊 Input: Vibration = {vibration_value} mm/s | Temperature = {temperature_value} °C | Time = {time_value} hours")

    risk_simulation.input['vibration'] = vibration_value
    risk_simulation.input['temperature'] = temperature_value
    risk_simulation.input['operation_time'] = time_value

    risk_simulation.compute()
    risk = risk_simulation.output['risk_level']

    print("\n📌 Membership Values (Simplified):")
    vibration_memberships = {
        "Low": fuzz.interp_membership(vibration.universe, vibration['low'].mf, vibration_value),
        "Medium": fuzz.interp_membership(vibration.universe, vibration['medium'].mf, vibration_value),
        "High": fuzz.interp_membership(vibration.universe, vibration['high'].mf, vibration_value)
    }

    temperature_memberships = {
        "Low": fuzz.interp_membership(temperature.universe, temperature['low'].mf, temperature_value),
        "Medium": fuzz.interp_membership(temperature.universe, temperature['medium'].mf, temperature_value),
        "High": fuzz.interp_membership(temperature.universe, temperature['high'].mf, temperature_value)
    }

    time_memberships = {
        "Short": fuzz.interp_membership(operation_time.universe, operation_time['short'].mf, time_value),
        "Moderate": fuzz.interp_membership(operation_time.universe, operation_time['moderate'].mf, time_value),
        "Long": fuzz.interp_membership(operation_time.universe, operation_time['long'].mf, time_value)
    }

    print(f" - Vibration: {vibration_memberships}")
    print(f" - Temperature: {temperature_memberships}")
    print(f" - Operation Time: {time_memberships}")

    strongest_rule = None
    highest_activation = 0.0

    for i, rule in enumerate(risk_control.rules):
        activation_level = 1.0

        if "low" in str(rule.antecedent) and vibration_memberships["Low"] > 0:
            activation_level = min(activation_level, vibration_memberships["Low"])
        if "medium" in str(rule.antecedent) and vibration_memberships["Medium"] > 0:
            activation_level = min(activation_level, vibration_memberships["Medium"])
        if "high" in str(rule.antecedent) and vibration_memberships["High"] > 0:
            activation_level = min(activation_level, vibration_memberships["High"])

        if activation_level > highest_activation:
            highest_activation = activation_level
            strongest_rule = f"Rule {i + 1} (Activation: {highest_activation:.4f})"

    print("\n✅ Strongest Active Rule:")
    if strongest_rule:
        print(f" - {strongest_rule}")
    else:
        print(" - No rule is active.")

    print(f"\n✅ Calculated Risk Level: {risk:.2f}")
    return risk


# **Section 5: Graph Display**
We define a clean function that displays the graph without affecting the calculation.

In [ ]:
def display_fuzzy_graphs(vibration_value=None, temperature_value=None, time_value=None, risk=None):
    plt.figure(figsize=(12, 8))
    plt.title("Fuzzy Control System - Risk Level")

    plt.plot(risk_level.universe, risk_level['low'].mf, label='Low', color='green')
    plt.plot(risk_level.universe, risk_level['moderate'].mf, label='Moderate', color='orange')
    plt.plot(risk_level.universe, risk_level['high'].mf, label='High', color='red')

    if risk is not None:
        plt.axvline(risk, color='blue', linestyle='--', label=f'Calculated Risk: {risk:.2f}')

    plt.xlabel("Risk Level")
    plt.ylabel("Membership")
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.show()


# **Section 6: Interactive User Interface**
Interactive controls for the user to test the fuzzy system dynamically.


In [ ]:
vibration_slider = widgets.FloatSlider(min=0, max=100, step=0.1, value=20, description='Vibration (mm/s)')
temperature_slider = widgets.FloatSlider(min=0, max=120, step=1, value=50, description='Temperature (°C)')
time_slider = widgets.FloatSlider(min=0, max=500, step=1, value=150, description='Time (h)')
run_button = widgets.Button(description="Run Fuzzy System", button_style='success')
output = widgets.Output()

display(vibration_slider, temperature_slider, time_slider, run_button, output)

def run_fuzzy_system(b):
    with output:
        clear_output()
        risk = calculate_risk(vibration_slider.value, temperature_slider.value, time_slider.value)
        display_fuzzy_graphs(risk=risk)

run_button.on_click(run_fuzzy_system)


# **Section 7: Automated Tests for Fuzzy System**
In this section, we will test the fuzzy system with a variety of input values:
- ✅ Low, Medium, and High Vibration values.
- ✅ Low, Medium, and High Temperature values.
- ✅ Short, Moderate, and Long Operation Time values.
- ✅ Clear and systematic test cases for accurate verification.


In [ ]:
def calculate_risk_for_test(vibration_value, temperature_value, time_value):
    risk_simulation.input['vibration'] = vibration_value
    risk_simulation.input['temperature'] = temperature_value
    risk_simulation.input['operation_time'] = time_value
    risk_simulation.compute()
    risk = risk_simulation.output['risk_level']
    return risk


def run_automated_tests():
    print("\n🔧 Running  Automated Tests for Fuzzy Control System...")
    test_cases = [
        {"name": "Test 1 - Low Risk", "vibration": 10, "temperature": 20, "time": 50, "expected": "Low"},
        {"name": "Test 2 - Moderate Risk", "vibration": 50, "temperature": 60, "time": 200, "expected": "Moderate"},
        {"name": "Test 3 - High Risk", "vibration": 90, "temperature": 110, "time": 450, "expected": "High"},
        {"name": "Test 4 - High Risk", "vibration": 60, "temperature": 100, "time": 400, "expected": "High"},
        {"name": "Test 5 - Moderate Risk", "vibration": 85, "temperature": 30, "time": 250, "expected": "Moderate"},
        {"name": "Test 6 - Moderate Risk", "vibration": 20, "temperature": 90, "time": 400, "expected": "Moderate"},
        {"name": "Test 7 - High Risk", "vibration": 100, "temperature": 120, "time": 500, "expected": "High"},
        {"name": "Test 8 - Moderate Risk", "vibration": 50, "temperature": 50, "time": 250, "expected": "Moderate"},
    ]

    results = []
    for test in test_cases:
        result = calculate_risk_for_test(test["vibration"], test["temperature"], test["time"])
        if result < 4:
            calculated_category = "Low"
        elif 4 <= result < 7:
            calculated_category = "Moderate"
        else:
            calculated_category = "High"

        test_result = "PASS" if calculated_category == test["expected"] else "FAIL"
        results.append({
            "Test Name": test["name"],
            "Vibration": test["vibration"],
            "Temperature": test["temperature"],
            "Time": test["time"],
            "Expected": test["expected"],
            "Calculated": calculated_category,
            "Risk Value": f"{result:.2f}",
            "Result": test_result
        })

    import pandas as pd
    results_df = pd.DataFrame(results)
    display(results_df)

    total_tests = len(results)
    passed_tests = results_df[results_df['Result'] == "PASS"].shape[0]
    failed_tests = total_tests - passed_tests
    success_rate = (passed_tests / total_tests) * 100

    print("\n✅ Test Summary:")
    print(f" - Total Tests: {total_tests}")
    print(f" - Passed Tests: {passed_tests}")
    print(f" - Failed Tests: {failed_tests}")
    print(f" - Success Rate: {success_rate:.2f}%")

    if failed_tests > 0:
        print("\n❌ Failed Tests:")
        display(results_df[results_df['Result'] == "FAIL"])

run_automated_tests()

# Fuzzy Vibration Control System

## Project Overview
This project is a **Fuzzy Control System for Monitoring Vibration in Industrial Equipment**, such as motors, pumps, or compressors. The system uses **Fuzzy Logic** to evaluate the risk level of equipment based on three main factors:
- **Vibration (0 - 100 mm/s)**
- **Temperature (0 - 120 °C)**
- **Operation Time (0 - 500 hours)**

The system calculates a **Risk Level (0 - 10)**, providing an automatic assessment of equipment status.

---

## Features
- Real-time calculation of equipment risk using Fuzzy Logic.
- Multiple input variables (Vibration, Temperature, Operation Time).
- Intuitive interactive interface using Google Colab.
- Clear and clean automated testing section.
- Easy-to-understand risk evaluation system with visual graphs.

---

## Technologies Used
- **Python (3.x)**
- **scikit-fuzzy** - Fuzzy Logic Control System Library
- **Matplotlib** - For Visual Graphs
- **ipywidgets** - Interactive User Interface (in Google Colab)
- **Pandas** - For Clear Test Result Tables

---



##  Project Structure (Detailed by Sections)

### **Section 1: Install and Import Required Libraries**
- **Install Libraries:**
  - `scikit-fuzzy` (Fuzzy Logic Control System)
  - `matplotlib` (Graph Visualization)
  - `ipywidgets` (Interactive User Interface)
  - `pandas` (Test Result Display)
- **Import Libraries:**
  - `numpy` (Numerical Calculations)
  - `skfuzzy` (Fuzzy Logic Control)
  - `matplotlib.pyplot` (Graph Display)
  - `ipywidgets` (Interactive UI Components)
  - `IPython.display` (Output Control)

---

### **Section 2: Defining Fuzzy Variables**
- **Vibration Variable (0 - 100 mm/s):**
  - Membership: Low, Medium, High
- **Temperature Variable (0 - 120 °C):**
  - Membership: Low, Medium, High
- **Operation Time Variable (0 - 500 hours):**
  - Membership: Short, Moderate, Long
- **Risk Level Variable (0 - 10):**
  - Membership: Low, Moderate, High

---

### **Section 3: Defining Fuzzy Rules and Controller**
- **High Risk Rules:**
  - Conditions where high values of Vibration, Temperature, or Time result in High Risk.
- **Moderate Risk Rules:**
  - Conditions that represent balanced or medium risk situations.
- **Low Risk Rules:**
  - Conditions where values are within safe operating ranges.

---

### **Section 4: Risk Calculation and Visualization**
- **Risk Calculation Function:**
  - Accepts inputs: Vibration, Temperature, and Operation Time.
  - Computes the Risk Level using Fuzzy Logic.
  - Displays the calculated risk and active rules.
- **Risk Graph Display:**
  - Visualizes the Risk Level using `matplotlib`.

---

### **Section 5: Graph Display (Visual Representation)**
- Displays a clear graphical representation of the Risk Levels.
- Risk levels are color-coded:
  - Green for Low Risk
  - Orange for Moderate Risk
  - Red for High Risk

---

### **Section 6: Interactive User Interface**
- **Sliders:**
  - User-adjustable sliders for:
    - Vibration (0 - 100 mm/s)
    - Temperature (0 - 120 °C)
    - Operation Time (0 - 500 hours)
- **Run Button:**
  - Calculates the risk based on slider values.
  - Displays the result directly below the sliders.

---

### **Section 7: Automated Tests for Fuzzy System**
- **Test Case Definitions:**
  - Multiple predefined test cases (Low, Moderate, High Risk scenarios).
  - Each test has expected values for comparison.
- **Test Result Display:**
  - Shows the results of each test case (Pass/Fail).
  - Displays a summary of all test cases (Total, Passed, Failed).

---

### **README.md (Project Documentation)**
- Clear documentation for project setup and usage.
- Project structure and explanation of each section.
- Instructions for running the project and automated tests.


---

## ⚡ Setup and Installation
1. Make sure you have **Python 3.x** installed.
2. Install the required libraries:
   ```bash
   pip install -U scikit-fuzzy matplotlib ipywidgets pandas